In [3]:
import evaluate

2023-09-29 17:09:24.116206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Evaluate使用指南
#### 查看支持的评估函数

In [6]:
evaluate.list_evaluation_modules(include_community=False, with_details=True)

#### 加载评估函数

In [ ]:
accuracy = evaluate.load("accuracy")

#### 查看函数说明

In [ ]:
print(accuracy.description)

#### 评估指标计算--全局计算

In [ ]:
accuracy = evaluate.load("accuracy")
results = accuracy.compute(references=[0,1,2,0,1,2], predictions=[0,1,1,2,1,0])
results

#### 评估指标计算--迭代计算

In [ ]:
accuracy = evaluate.load("accuracy")
for ref, pred in zip([0,1,0,1], [1,0,0,1]):
    accuracy.add(references=ref, predictions=pred)
accuracy.compute()

In [ ]:
accuracy = evaluate.load("accuracy")
for refs, preds in zip([[0,1],[0,1]],[[1,0],[0,1]]):
    accuracy.add_batch(references=refs, predictions=preds)
accuracy.compute()

#### 多个评估指标计算

In [ ]:
clf_metrics = evaluate.combine(["accuracy", "f1", "recall", "precision"])
clf_metrics

In [ ]:
clf_metrics.compute(predictions=[0,1,0], references=[0,1,1])

#### 评估结果对比可视化

In [ ]:
from evaluate.visualization import radar_plot # 目前只支持雷达图

data = [
    {"accuracy":0.99,"precision":0.8, "f1":0.95, "latency_in_seconds":33.6},
    {"accuracy":0.98,"precision":0.87, "f1":0.91, "latency_in_seconds":11.2},
    {"accuracy":0.98,"precision":0.78, "f1":0.88, "latency_in_seconds":87.6},
    {"accuracy":0.88,"precision":0.78, "f1":0.81, "latency_in_seconds":101.6},
]

model_names = ["Model 1", "Model 2", "Model 3", "Model 4"]

In [ ]:
plot = radar_plot(data=data, model_names=model_names)

## 模型微调训练

In [ ]:
# Step 7
import evaluate
clf_metrics = evaluate.combine(["accuracy","f1"])


def evaluate():
    model.eval()
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            clf_metrics.add_batch(predictions=pred.long(), references=batch["labels"].long())
    return clf_metrics.compute()

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, {aclf}")

train()